In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

#TODO - Lembrar de adicionar uma extensão de vpn ao firefox
#TODO - Resolver captcha, resolver proxy

def adicionar_conta_txt(conta: str):
    # Nome do arquivo de saída
    arquivo_txt = f'contas/contas_criadas.txt'

    # Garantir que o diretório 'contas/' exista
    os.makedirs(os.path.dirname(arquivo_txt), exist_ok=True)

    # Escrever no arquivo .txt
    with open(arquivo_txt, 'a') as f:
        f.write(conta + '\n')

    print(f'Conta gravada em {arquivo_txt}')

def configurar_proxy(options, proxy: str):
    options.set_proxy(proxy)
    return options

def validar_caminho_gecko(path: str):
    # Verifica se o GeckoDriver existe no caminho especificado
    if not os.path.isfile(path):
        raise FileNotFoundError(f"O arquivo '{path}' não foi encontrado. Verifique o caminho.")
    
def cadastrar_conta_papaya(driver, email):
    # Navega para a página de registro
    driver.get('https://www.papayaplay.com/account/signup.do')

    # Encontra os campos do formulário e preenche as informações
    campo_usuario = driver.find_element(By.NAME, 'userid')
    campo_email = driver.find_element(By.ID, 'equalEmail')
    campo_confirma_email = driver.find_element(By.ID, 'confirmEmail')
    campo_senha = driver.find_element(By.NAME, 'pwd')
    campo_checkbox = driver.find_element(By.ID, 'custCheckbox')

    usuario_conta = email.split('@')[0]
    # Preenche os campos do formulário com os dados atuais
    campo_usuario.send_keys(usuario_conta)
    campo_email.send_keys(email)
    campo_confirma_email.send_keys(email)
    campo_senha.send_keys(usuario_conta)
    if not campo_checkbox.is_selected():
        driver.execute_script("arguments[0].click();", campo_checkbox)

    esperar_resolucao_captcha(driver)

    # Espera até que o botão de envio esteja presente e visível
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[2]/div/form/button'))
    )

    driver.execute_script("arguments[0].scrollIntoView(true);", button)
    button.click()

    return usuario_conta

def obter_email_temporario(driver, site_email_temporario: str):
    # Navega para a página de geração de email temporário
    driver.get(site_email_temporario)

    # 1. Extrai o email gerado
    email_element = driver.find_element(By.ID, 'email-input')
    email_temporario = email_element.get_attribute('value')
    print(f'Email gerado: {email_temporario}')

    return email_temporario

def clicar_email(driver):
    # XPath do tbody
    tbody_xpath = '/html/body/main/div[1]/div[3]/div[1]/table/tbody'
    # Localiza o tbody e depois o tr específico dentro dele
    tbody_element = driver.find_element(By.XPATH, tbody_xpath)

    # Encontra todos os elementos <tr> dentro do tbody
    tr_elements = tbody_element.find_elements(By.XPATH, './tr')

    # Loop para encontrar o <tr> com o ID dinâmico e extrair o ID
    for tr_element in tr_elements:
        # Tenta extrair o ID do elemento <tr>
        try:
            element_id = tr_element.get_attribute('id')
            if element_id:
                print(f'ID encontrado: {element_id}')
                # Construa o XPath usando o ID encontrado
                dynamic_tr_xpath = f'//*[@id="{element_id}"]'
                
                # Executa o script JavaScript para clicar no elemento
                script = """
                    var element = arguments[0];
                    if (element) {
                        element.click();
                    }
                """
                driver.execute_script(script, tr_element)
                
                # Se o elemento foi clicado, saia do loop
                break
        except Exception as e:
            print(f'Erro ao encontrar o ID: {e}')

    # Aguarda um pouco para garantir que a ação foi executada
    WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.XPATH, f'//*[@id="{element_id}"]'))  # Ajuste o XPath conforme necessário
    )

def clicar_link(driver):
    # XPath do elemento que contém o Shadow DOM
    shadow_host_xpath = '//*[@id="body"]'

    # Localiza o elemento que contém o Shadow DOM
    shadow_host = driver.find_element(By.XPATH, shadow_host_xpath)

    # Script JavaScript para acessar o Shadow DOM e encontrar todos os links
    script_shadow = """
        var shadowHost = arguments[0];
        var shadowRoot = shadowHost.shadowRoot;
        if (shadowRoot) {
            return Array.from(shadowRoot.querySelectorAll('a')).map(a => a.href);
        }
        return [];
    """
    # Executa o script JavaScript para obter todos os links dentro do Shadow DOM
    links = driver.execute_script(script_shadow, shadow_host)

    # Verifica se há links encontrados
    if links:
        first_link = links[0]
        print(f"Primeiro link encontrado: {first_link}")

        # Navega para o primeiro link encontrado
        driver.get(first_link)
    else:
        print("Nenhum link encontrado.")

def validar_email_apos_criar_conta_papaya(driver):
    clicar_email(driver)
    clicar_link(driver)

def validar_criacao_conta_papaya(driver):
    body_element = driver.find_element(By.ID, "portal")
    h2_elements = body_element.find_elements(By.XPATH, './h2')

    for h2_element in h2_elements:
        try:
            h2_value = h2_element.get_attribute('value')
            if h2_value == "Sign Up Completed":
                print("Conta cadastrada com sucesso")
                break
        except Exception as e:
            print(f'Erro ao cadastrar conta')
            
def esperar_resolucao_captcha(driver):
    # Aguarda até que o iframe do reCAPTCHA esteja presente e mude para ele
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe[title='reCAPTCHA']")))

    # Aguarda até que o elemento do reCAPTCHA esteja presente
    captcha = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "recaptcha-anchor")))

    #Loop infinito até que o captcha seja resolvido
    while True:
        time.sleep(2)
        # Verifica se o elemento tem a classe que indica que o reCAPTCHA foi verificado
        class_name = "recaptcha-checkbox-checked"
        if class_name in captcha.get_attribute("class"):
            print("reCAPTCHA foi verificado com sucesso!")
            break
        else:
            print("reCAPTCHA ainda não foi verificado.")

    # Volta para o contexto principal da página
    driver.switch_to.default_content()

def obter_profile(path):
    # Carrega o perfil do Firefox
    profile = webdriver.FirefoxProfile(path)

    return profile

In [ ]:
site_email_temporario = 'https://www.invertexto.com/gerador-email-temporario'

gecko_path = os.path.join('geckodriver.exe')
validar_caminho_gecko(gecko_path)

# Configura as opções do Firefox
options = Options()

#options = configurar_proxy("????????")

profile = obter_profile(r'C:\Users\Kaio\AppData\Roaming\Mozilla\Firefox\Profiles\i0bokd72.default-release')

# Inicializa o WebDriver para o Firefox com o caminho do GeckoDriver, o perfil e as opções
driver = webdriver.Firefox(executable_path=gecko_path, firefox_profile=profile, options=options)

# Loop para criar contas com diferentes usuários e senhas
quantidade_contas_criacao = 5
for i in range(quantidade_contas_criacao):
    email_temporario = obter_email_temporario(driver, site_email_temporario)

    usuario_conta = cadastrar_conta_papaya(driver, email_temporario)

    # Volta para a página de email temporário para verificar o email
    #https://www.invertexto.com/gerador-email-temporario?email=mahayuba7629@uorak.com
    driver.get(site_email_temporario + f"?email={email_temporario}")

    # Espera um pouco para receber o email de confirmação
    time.sleep(10)

    validar_email_apos_criar_conta_papaya(driver)
    validar_criacao_conta_papaya(driver)

    adicionar_conta_txt(usuario_conta)

# Fecha o navegador ao final do processo
driver.quit()